In [30]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd

In [31]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
servicekey = 'GCyZSspoE7iPOuN+4UKO10fOhJyXmQhnhGwke3KcD8v2qaWYLQkoGLNn7vTwbctZkR/3uEECJAykpO4UJxUwTQ=='
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : servicekey, 
                               quote_plus('pageNo') : '1', 
                               quote_plus('numOfRows') : '10', 
                               quote_plus('startCreateDt') : '20200310', 
                               quote_plus('endCreateDt') : '20200315' })

In [32]:
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read()
print(response_body)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>3.2396602365</accDefRate><accExamCnt>268212</accExamCnt><accExamCompCnt>251940</accExamCompCnt><careCnt>7300</careCnt><clearCnt>834</clearCnt><createDt>2020-03-15 10:01:22.22</createDt><deathCnt>75</deathCnt><decideCnt>8162</decideCnt><examCnt>16272</examCnt><resutlNegCnt>243778</resutlNegCnt><seq>74</seq><stateDt>20200315</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-15 10:01:22.22</updateDt></item><item><accDefRate>3.3180003364</accDefRate><accExamCnt>261335</accExamCnt><accExamCompCnt>243701</accExamCompCnt><careCnt>7402</careCnt><clearCnt>714</clearCnt><createDt>2020-03-14 10:29:11.11</createDt><deathCnt>72</deathCnt><decideCnt>8086</decideCnt><examCnt>17634</examCnt><resutlNegCnt>235615</resutlNegCnt><seq>73</seq><stateDt>20200314</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-14 10:29:1

In [33]:
print(type(response_body))

print(request)

<class 'bytes'>


In [34]:
# ServiceKey : 공공데이터포털에서 발급받은 서비스키
# startCreateDt : 데이터 생성일 시작범위
# endCreateDt : 데이터 생성일 종료범위
params = {"ServiceKey" : parse.unquote(servicekey),
         'startCreateDt' : 20200119,
         "endCreateDt" : 20210901}

In [35]:
url = url + "?"
res = requests.get(url, params = params)
soup = bs(res.text, 'lxml')

In [36]:
print(soup)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item><accdefrate>2.0486116135</accdefrate><accexamcnt>13152138</accexamcnt><accexamcompcnt>12371159</accexamcompcnt><carecnt>26826</carecnt><clearcnt>224319</clearcnt><createdt>2021-09-01 09:37:02.395</createdt><deathcnt>2292</deathcnt><decidecnt>253437</decidecnt><examcnt>780979</examcnt><resutlnegcnt>12117722</resutlnegcnt><seq>621</seq><statedt>20210901</statedt><statetime>00:00</statetime><updatedt>2021-09-19 09:24:03.195</updatedt></item><item><accdefrate>2.0368178478</accdefrate><accexamcnt>13094711</accexamcnt><accexamcompcnt>12343470</accexamcompcnt><carecnt>27428</carecnt><clearcnt>221701</clearcnt><createdt>2021-08-31 09:38:37.509</createdt><deathcnt>2285</deathcnt><decidecnt>251414</decidecnt><examcnt>751241</examcnt><resutlnegcnt>12092056</resutlnegcnt><seq>620</seq><statedt>20210831</statedt><statetime>00:00<

In [37]:
print(soup.find_all("decidecnt")[0].text)

253437


In [38]:
decidecnt_data = soup.find_all("decidecnt")
statedt_data = soup.find_all("statedt")

In [39]:
decidecnt_list = []
statedt_list = []
daycnt_list = []

In [40]:
for one in range(len(decidecnt_data)) :
    decidecnt_list.append(int(decidecnt_data[one].text))
    if one != len(decidecnt_data) - 1 :
        statedt_list.append(statedt_data[one].text)

In [42]:
print("decidecnt len :", len(decidecnt_list))
print("statedt len :", len(statedt_list))

decidecnt len : 580
statedt len : 579


In [43]:
for i in range(len(decidecnt_data)) :
    print(decidecnt_list[i]) # 2381

for i in range(len(statedt_list)) :
    print(statedt_list[i])

for i in range(0, len(decidecnt_data) - 1) :
    print(decidecnt_list[i], decidecnt_list[i + 1])
    daycnt_list.append(decidecnt_list[i] - decidecnt_list[i + 1])

for i in daycnt_list :
    print(i)

253437
251414
250044
248557
246942
245149
243312
241431
239277
237770
236353
234727
232850
230800
228648
226845
225473
223919
222103
220175
218186
216199
213978
212441
210950
209222
207399
205695
203920
202195
200995
199777
198335
196797
195087
193414
191519
190156
188838
187351
185723
184094
182253
180472
179194
177943
176490
175041
173506
171907
170294
169144
168044
166720
165342
164026
162751
161540
160794
160083
159340
158546
157721
156960
156166
155571
155070
154456
153788
153154
152544
151899
151505
151148
150719
150237
149730
149190
148645
148272
147873
147421
146856
146300
145690
145088
144634
144149
143593
142849
142154
141473
140796
140337
139907
139427
138894
138307
137678
136973
136457
135927
135342
134676
134115
133469
132815
132287
131668
131058
130377
129630
128915
128281
127771
127308
126744
126043
125518
124944
124268
123727
123239
122633
122006
121345
120666
119897
119385
118886
118242
117457
116660
115925
72328
115194
114645
114113
113442
112784
112111
111413
110682


16346 16058
16058 15761
15761 15515
15515 15318
15318 15039
15039 14873
14873 14770
14770 14714
14714 14660
14660 14626
14626 14598
14598 14562
14562 14519
14519 14499
14499 14456
14456 14423
14423 14389
14389 14366
14366 14336
14336 14305
14305 14269
14269 14251
14251 14203
14203 14175
14175 14150
14150 14092
14092 13979
13979 13938
13938 13879
13879 13816
13816 13771
13771 13745
13745 13711
13711 13672
13672 13612
13612 13551
13551 13512
13512 13479
13479 13417
13417 13373
13373 13338
13338 13293
13293 13243
13243 13181
13181 13137
13137 13089
13089 13028
13028 12965
12965 12904
12904 12850
12850 12799
12799 12757
12757 12715
12715 12653
12653 12602
12602 12563
12563 12535
12535 12484
12484 12438
12438 12421
12421 12373
12373 12306
12306 12257
12257 12198
12198 12155
12155 12121
12121 12084
12084 12050
12050 12002
12002 11946
11946 11901
11901 11851
11851 11813
11813 11776
11776 11719
11719 11668
11668 11629
11629 11590
11590 11541
11541 11503
11503 11468
11468 11441
11441 11402
1140

In [44]:
# 27 - 26 = 2381 -> 26일 데이터
corona_dic = {"날짜" : statedt_list, "일일 확진자수" : daycnt_list}
corona_df = pd.DataFrame(data = corona_dic)
corona_df

,날짜,일일 확진자수
0,20210901,2023
1,20210831,1370
2,20210830,1487
3,20210829,1615
4,20210828,1793
...,...,...
574,20200205,19
575,20200204,-15
576,20200203,13
577,20200202,2


In [45]:
corona_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   날짜       579 non-null    object
 1   일일 확진자수  579 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.2+ KB


In [48]:
corona_df

,날짜,일일 확진자수
0,20210901,2023
1,20210831,1370
2,20210830,1487
3,20210829,1615
4,20210828,1793
5,20210827,1837
6,20210826,1881
7,20210825,2154
8,20210824,1507
9,20210823,1417


In [49]:
corona_df.loc[corona_df["일일 확진자수"] < 0]

,날짜,일일 확진자수
134,20210117,-42866
266,20201208,-670
575,20200204,-15


In [27]:
len(decidecnt_list)

580

In [50]:
for i in range(260, 270) :
    print(decidecnt_list[i])

44356
43476
42758
41728
40778
40089
38739
39409
38152
37537
